In [4]:
import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):     #nn.Module을 상속해서 인공신경망을 만드는 것을 피하게 할 수 있다.
    def __init__(self):
        super().__init__()              #super().__init__()이라는 코드가 다른 클래스의 속성 및 메소드를 자동으로 불러와 해당 클래스에서도 사용이 가능하도록 해준다.
                                        #super()는 부모 클래스의 인스턴스를 반환하는 함수. __init__()는 부모 클래스의 초기화 메서드.
        self.linear=nn.Linear(3,1)      #선형 계층(Linear Layer)을 정의. 입력차원은 3, 출력차원은 1
                                        #self.linear는 이 선형 계층을 MultivariateLinearRegressionModel 클래스의 속성으로 저장        
                                        #nn.Linear는 in_features(입력 특성의 수)와 out_features(출력 특성의 수)에 맞춰 W와 b를 자동으로 초기화

    def forward(self,x):
        return self.linear(x)

In [ ]:
from torch.nn.functional import F

# minibatch 를 이용한 gradient descent

In [21]:
#pytorch Dataset
# 여기서 x와 y에 대해 정의를 했음.
import torch                                          
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data=torch.FloatTensor([[73, 80, 75],
                           [93,88,93],
                           [89,91,90],
                           [96,98,100],
                           [73,66,70]])
        
        self.y_data=torch.FloatTensor([[152],[185],[180],[196],[142]])

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self,idx):
        x=torch.FloatTensor(self.x_data[idx])
        y=torch.FloatTensor(self.y_data[idx])

        return x,y

dataset=CustomDataset()
model=MultivariateLinearRegressionModel()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-5)

In [22]:
#이걸 이제 mini batch 처리를 함.

from torch.utils.data import DataLoader

dataloader=DataLoader(
    dataset,
    batch_size=2,               #보통 batch_size는 2의 제곱수로 둔다.
    shuffle=True
)

In [23]:
import torch.nn.functional as F


nb_epochs=20

for epoch in range(nb_epochs+1):                                #epoch이 돌아갈 때마다 아래의 for문에서 dataloader가 호출이 되는데
    for batch_idx, samples in enumerate(dataloader):            #2개 2개 1개 씩 호출이 된다.                                            
        x_train, y_train= samples             #예시: x_train = [[73, 80, 75],    [93, 88, 93]]
                                                    #y_train = [[152], [185]]

        predictions=model(x_train)

        cost=F.mse_loss(predictions, y_train)

        optimizer.zero_grad()
        cost.backward() 
        optimizer.step()
        
        print('epoch {:4d}/{} batch: {}/{}, cost: {:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()))


epoch    0/20 batch: 1/3, cost: 17146.664062
epoch    0/20 batch: 2/3, cost: 12955.142578
epoch    0/20 batch: 3/3, cost: 2748.992188
epoch    1/20 batch: 1/3, cost: 457.923676
epoch    1/20 batch: 2/3, cost: 306.978516
epoch    1/20 batch: 3/3, cost: 73.785416
epoch    2/20 batch: 1/3, cost: 16.777370
epoch    2/20 batch: 2/3, cost: 2.841071
epoch    2/20 batch: 3/3, cost: 4.225943
epoch    3/20 batch: 1/3, cost: 0.900907
epoch    3/20 batch: 2/3, cost: 0.137768
epoch    3/20 batch: 3/3, cost: 0.111179
epoch    4/20 batch: 1/3, cost: 0.152790
epoch    4/20 batch: 2/3, cost: 0.625106
epoch    4/20 batch: 3/3, cost: 0.045909
epoch    5/20 batch: 1/3, cost: 0.391222
epoch    5/20 batch: 2/3, cost: 0.385556
epoch    5/20 batch: 3/3, cost: 0.230323
epoch    6/20 batch: 1/3, cost: 0.117620
epoch    6/20 batch: 2/3, cost: 0.098170
epoch    6/20 batch: 3/3, cost: 1.255620
epoch    7/20 batch: 1/3, cost: 0.246526
epoch    7/20 batch: 2/3, cost: 0.234694
epoch    7/20 batch: 3/3, cost: 1.045645

batch가 1/3, 2/3, 3/3 이 되는 이유:

총 데이터 개수는 5개입니다.<br>
배치 크기는 2입니다.<br>
5개를 2개씩 나누면 첫 번째와 두 번째 배치는 2개의 샘플로, 세 번째 배치는 나머지 1개로 처리됩니다.<br>